In [3]:
import requests
import time
from datetime import date, timedelta  # datetime modülünden date ve timedelta sınıflarını içe aktarın

# Başlangıç ve bitiş tarihlerini belirtin
start_date = date(2023, 7, 1)
end_date = date(2023, 9, 1)

# URL'leri saklamak için bir liste oluşturun
url_list = []

# Toplam site sayısı
total_sites = (end_date - start_date).days + 1

# Tarih aralığını döngüyle geçin
current_date = start_date
while current_date <= end_date:
    # URL'yi oluşturun
    url = f"https://tr.weatherspark.com/h/d/95434/{current_date.year}/{current_date.month}/{current_date.day}/{current_date.strftime('%d-%B-%Y')}-tarihinde-in%C4%B0stanbul-T%C3%BCrkiye-Ortalama-Hava-Durumu#Figures-Temperature"

    # URL'yi listeye ekleyin
    url_list.append(url)

    # Sayfa bilgisi
    percent_done = len(url_list) / total_sites * 100
    print(f"Toplam {len(url_list)} link alındı. {total_sites - len(url_list)} tane kaldı.")

    # 10 saniye bekleme
    time.sleep(5)

    # Bir sonraki tarihe geçin
    current_date += timedelta(days=1)

# Toplam site sayısını yazdırın
print(f"Toplam {len(url_list)} site alındı.")

Toplam 1 link alındı. 62 tane kaldı.
Toplam 2 link alındı. 61 tane kaldı.
Toplam 3 link alındı. 60 tane kaldı.
Toplam 4 link alındı. 59 tane kaldı.
Toplam 5 link alındı. 58 tane kaldı.
Toplam 6 link alındı. 57 tane kaldı.
Toplam 7 link alındı. 56 tane kaldı.
Toplam 8 link alındı. 55 tane kaldı.
Toplam 9 link alındı. 54 tane kaldı.
Toplam 10 link alındı. 53 tane kaldı.
Toplam 11 link alındı. 52 tane kaldı.
Toplam 12 link alındı. 51 tane kaldı.
Toplam 13 link alındı. 50 tane kaldı.
Toplam 14 link alındı. 49 tane kaldı.
Toplam 15 link alındı. 48 tane kaldı.
Toplam 16 link alındı. 47 tane kaldı.
Toplam 17 link alındı. 46 tane kaldı.
Toplam 18 link alındı. 45 tane kaldı.
Toplam 19 link alındı. 44 tane kaldı.
Toplam 20 link alındı. 43 tane kaldı.
Toplam 21 link alındı. 42 tane kaldı.
Toplam 22 link alındı. 41 tane kaldı.
Toplam 23 link alındı. 40 tane kaldı.
Toplam 24 link alındı. 39 tane kaldı.
Toplam 25 link alındı. 38 tane kaldı.
Toplam 26 link alındı. 37 tane kaldı.
Toplam 27 link alındı

In [4]:
url_list

['https://tr.weatherspark.com/h/d/95434/2023/7/1/01-July-2023-tarihinde-in%C4%B0stanbul-T%C3%BCrkiye-Ortalama-Hava-Durumu#Figures-Temperature',
 'https://tr.weatherspark.com/h/d/95434/2023/7/2/02-July-2023-tarihinde-in%C4%B0stanbul-T%C3%BCrkiye-Ortalama-Hava-Durumu#Figures-Temperature',
 'https://tr.weatherspark.com/h/d/95434/2023/7/3/03-July-2023-tarihinde-in%C4%B0stanbul-T%C3%BCrkiye-Ortalama-Hava-Durumu#Figures-Temperature',
 'https://tr.weatherspark.com/h/d/95434/2023/7/4/04-July-2023-tarihinde-in%C4%B0stanbul-T%C3%BCrkiye-Ortalama-Hava-Durumu#Figures-Temperature',
 'https://tr.weatherspark.com/h/d/95434/2023/7/5/05-July-2023-tarihinde-in%C4%B0stanbul-T%C3%BCrkiye-Ortalama-Hava-Durumu#Figures-Temperature',
 'https://tr.weatherspark.com/h/d/95434/2023/7/6/06-July-2023-tarihinde-in%C4%B0stanbul-T%C3%BCrkiye-Ortalama-Hava-Durumu#Figures-Temperature',
 'https://tr.weatherspark.com/h/d/95434/2023/7/7/07-July-2023-tarihinde-in%C4%B0stanbul-T%C3%BCrkiye-Ortalama-Hava-Durumu#Figures-Temper

In [6]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import re

In [14]:
# Her URL için sonuçları saklamak için bir liste oluşturun
all_results = []

# URL'leri döngü ile gezip sonuçları toplayın
for url in url_list:
    result = {}  # Her URL için yeni bir sonuç sözlüğü oluşturun

    # URL'yi getirin ve sayfanın içeriğini çekin
    headers = {
        "User-Agent": "Safari/537.36",
        "Referer": "https://tr.weatherspark.com/"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
try:
    # İlk olarak "00:20" saatine ait metni bulun ve temizleyin
    result['00:20'] = soup.find('tr', class_='metar-00-50 History-MetarReports-row metar-00-20').text.strip()
    
    # Ardından "Çiyleşme Noktası" metnini bulun ve derece değerini çıkarın
    dew_point_cell = soup.find("tr", class_='table_no_border metar-00-50 History-MetarReports-row metar-00-20').find("td", {"class": "r i nowrap"}, string=re.compile("Çiyleşme Noktası"))
    degree_value = dew_point_cell.findNext("td").text.strip()

    # Derece değerini elde edin
    match = re.search(r'(\d+°C)', degree_value)
    if match:
        degree_value = match.group(1)
    else:
        degree_value = "NaN"

    result['00:20_nem'] = degree_value  # Farklı bir anahtar kullanarak derece değerini saklayın
    
except AttributeError:
    result['00:20'] = np.nan  # "00:20" metni bulunamazsa
    result['00:20_nem'] = "NaN"  # "Çiyleşme Noktası" metni veya derece değeri bulunamazsa
    
try:
    # İlk olarak "02:20" saatine ait metni bulun ve temizleyin
    result['02:20'] = soup.find('tr', class_='table_strong_border metar-02-50 History-MetarReports-row metar-02-20').text.strip()
    
    # Ardından "Çiyleşme Noktası" metnini bulun ve derece değerini çıkarın
    dew_point_cell = soup.find("tr", class_='table_no_border metar-02-50 History-MetarReports-row metar-02-20').find("td", {"class": "r i nowrap"}, string=re.compile("Çiyleşme Noktası"))
    degree_value = dew_point_cell.findNext("td").text.strip()

    # Derece değerini elde edin
    match = re.search(r'(\d+°C)', degree_value)
    if match:
        degree_value = match.group(1)
    else:
        degree_value = "NaN"

    result['02:20_nem'] = degree_value  # Farklı bir anahtar kullanarak derece değerini saklayın

except AttributeError:
    result['02:20'] = np.nan  # "02:20" metni bulunamazsa
    result['02:20_nem'] = "NaN"  # "Çiyleşme Noktası" metni veya derece değeri bulunamazsa
    
try:
    # İlk olarak "04:20" saatine ait metni bulun ve temizleyin
    result['04:20'] = soup.find('tr', class_='table_strong_border metar-04-50 History-MetarReports-row metar-04-20').text.strip()
    
    # Ardından "Çiyleşme Noktası" metnini bulun ve derece değerini çıkarın
    dew_point_cell = soup.find("tr", class_='table_no_border metar-04-50 History-MetarReports-row metar-04-20').find("td", {"class": "r i nowrap"}, string=re.compile("Çiyleşme Noktası"))
    degree_value = dew_point_cell.findNext("td").text.strip()

    # Derece değerini elde edin
    match = re.search(r'(\d+°C)', degree_value)
    if match:
        degree_value = match.group(1)
    else:
        degree_value = "NaN"

    result['04:20_nem'] = degree_value  # Farklı bir anahtar kullanarak derece değerini saklayın

except AttributeError:
    result['04:20'] = np.nan  # "04:20" metni bulunamazsa
    result['04:20_nem'] = "NaN"  # "Çiyleşme Noktası" metni veya derece değeri bulunamazsa
    
try:
    # İlk olarak "06:20" saatine ait metni bulun ve temizleyin
    result['06:20'] = soup.find('tr', class_='metar-06-50 History-MetarReports-row metar-06-20').text.strip()
    
    # Ardından "Çiyleşme Noktası" metnini bulun ve derece değerini çıkarın
    dew_point_cell = soup.find("tr", class_='table_no_border metar-06-50 History-MetarReports-row metar-06-20').find("td", {"class": "r i nowrap"}, string=re.compile("Çiyleşme Noktası"))
    degree_value = dew_point_cell.findNext("td").text.strip()

    # Derece değerini elde edin
    match = re.search(r'(\d+°C)', degree_value)
    if match:
        degree_value = match.group(1)
    else:
        degree_value = "NaN"

    result['06:20_nem'] = degree_value  # Farklı bir anahtar kullanarak derece değerini saklayın

except AttributeError:
    result['06:20'] = np.nan  # "06:20" metni bulunamazsa
    result['06:20_nem'] = "NaN"  # "Çiyleşme Noktası" metni veya derece değeri bulunamazsa
    

    
try:
    # İlk olarak "08:20" saatine ait metni bulun ve temizleyin
    result['08:20'] = soup.find('tr', class_='table_strong_border metar-08-50 History-MetarReports-row metar-08-20').text.strip()
    
    # Ardından "Çiyleşme Noktası" metnini bulun ve derece değerini çıkarın
    dew_point_cell = soup.find("tr", class_='table_no_border metar-08-50 History-MetarReports-row metar-08-20').find("td", {"class": "r i nowrap"}, string=re.compile("Çiyleşme Noktası"))
    degree_value = dew_point_cell.findNext("td").text.strip()

    # Derece değerini elde edin
    match = re.search(r'(\d+°C)', degree_value)
    if match:
        degree_value = match.group(1)
    else:
        degree_value = "NaN"

    result['08:20_nem'] = degree_value  # Farklı bir anahtar kullanarak derece değerini saklayın

except AttributeError:
    result['08:20'] = np.nan  # "08:20" metni bulunamazsa
    result['08:20_nem'] = "NaN"  # "Çiyleşme Noktası" metni veya derece değeri bulunamazsa
    
    
try:
    # İlk olarak "10:20" saatine ait metni bulun ve temizleyin
    result['10:20'] = soup.find('tr', class_='table_strong_border metar-10-50 History-MetarReports-row metar-10-20').text.strip()
    
    # Ardından "Çiyleşme Noktası" metnini bulun ve derece değerini çıkarın
    dew_point_cell = soup.find("tr", class_='table_no_border metar-10-50 History-MetarReports-row metar-10-20').find("td", {"class": "r i nowrap"}, string=re.compile("Çiyleşme Noktası"))
    degree_value = dew_point_cell.findNext("td").text.strip()

    # Derece değerini elde edin
    match = re.search(r'(\d+°C)', degree_value)
    if match:
        degree_value = match.group(1)
    else:
        degree_value = "NaN"

    result['10:20_nem'] = degree_value  # Farklı bir anahtar kullanarak derece değerini saklayın

except AttributeError:
    result['10:20'] = np.nan  # "10:20" metni bulunamazsa
    result['10:20_nem'] = "NaN"  # "Çiyleşme Noktası" metni veya derece değeri bulunamazsa
    
    
try:
    # İlk olarak "12:20" saatine ait metni bulun ve temizleyin
    result['12:20'] = soup.find('tr', class_='metar-12-50 History-MetarReports-row metar-12-20').text.strip()
    
    # Ardından "Çiyleşme Noktası" metnini bulun ve derece değerini çıkarın
    dew_point_cell = soup.find("tr", class_='table_no_border metar-12-50 History-MetarReports-row metar-12-20').find("td", {"class": "r i nowrap"}, string=re.compile("Çiyleşme Noktası"))
    degree_value = dew_point_cell.findNext("td").text.strip()

    # Derece değerini elde edin
    match = re.search(r'(\d+°C)', degree_value)
    if match:
        degree_value = match.group(1)
    else:
        degree_value = "NaN"

    result['12:20_nem'] = degree_value  # Farklı bir anahtar kullanarak derece değerini saklayın

except AttributeError:
    result['12:20'] = np.nan  # "12:20" metni bulunamazsa
    result['12:20_nem'] = "NaN"  # "Çiyleşme Noktası" metni veya derece değeri bulunamazsa
    
    
try:
    # İlk olarak "14:20" saatine ait metni bulun ve temizleyin
    result['14:20'] = soup.find('tr', class_='History-MetarReports-superseded metar-14-50 History-MetarReports-row metar-14-20').text.strip()
    
    # Ardından "Çiyleşme Noktası" metnini bulun ve derece değerini çıkarın
    dew_point_cell = soup.find("tr", class_='table_no_border metar-14-50 History-MetarReports-row metar-14-20').find("td", {"class": "r i nowrap"}, string=re.compile("Çiyleşme Noktası"))
    degree_value = dew_point_cell.findNext("td").text.strip()

    # Derece değerini elde edin
    match = re.search(r'(\d+°C)', degree_value)
    if match:
        degree_value = match.group(1)
    else:
        degree_value = "NaN"

    result['14:20_nem'] = degree_value  # Farklı bir anahtar kullanarak derece değerini saklayın

except AttributeError:
    result['14:20'] = np.nan  # "14:20" metni bulunamazsa
    result['14:20_nem'] = "NaN"  # "Çiyleşme Noktası" metni veya derece değeri bulunamazsa
    
    
try:
    # İlk olarak "16:20" saatine ait metni bulun ve temizleyin
    result['16:20'] = soup.find('tr', class_='table_strong_border metar-16-50 History-MetarReports-row metar-16-20').text.strip()
    
    # Ardından "Çiyleşme Noktası" metnini bulun ve derece değerini çıkarın
    dew_point_cell = soup.find("tr", class_='table_no_border metar-16-50 History-MetarReports-row metar-16-20').find("td", {"class": "r i nowrap"}, string=re.compile("Çiyleşme Noktası"))
    degree_value = dew_point_cell.findNext("td").text.strip()

    # Derece değerini elde edin
    match = re.search(r'(\d+°C)', degree_value)
    if match:
        degree_value = match.group(1)
    else:
        degree_value = "NaN"

    result['16:20_nem'] = degree_value  # Farklı bir anahtar kullanarak derece değerini saklayın

except AttributeError:
    result['16:20'] = np.nan  # "16:20" metni bulunamazsa
    result['16:20_nem'] = "NaN"  # "Çiyleşme Noktası" metni veya derece değeri bulunamazsa
    
    
try:
    # İlk olarak "18:20" saatine ait metni bulun ve temizleyin
    result['18:20'] = soup.find('tr', class_='metar-18-50 History-MetarReports-row metar-18-20').text.strip()
    
    # Ardından "Çiyleşme Noktası" metnini bulun ve derece değerini çıkarın
    dew_point_cell = soup.find("tr", class_='table_no_border metar-18-50 History-MetarReports-row metar-18-20').find("td", {"class": "r i nowrap"}, string=re.compile("Çiyleşme Noktası"))
    degree_value = dew_point_cell.findNext("td").text.strip()

    # Derece değerini elde edin
    match = re.search(r'(\d+°C)', degree_value)
    if match:
        degree_value = match.group(1)
    else:
        degree_value = "NaN"

    result['18:20_nem'] = degree_value  # Farklı bir anahtar kullanarak derece değerini saklayın

except AttributeError:
    result['18:20'] = np.nan  # "18:20" metni bulunamazsa
    result['18:20_nem'] = "NaN"  # "Çiyleşme Noktası" metni veya derece değeri bulunamazsa
    
    
try:
    # İlk olarak "20:20" saatine ait metni bulun ve temizleyin
    result['20:20'] = soup.find('tr', class_='table_strong_border metar-20-50 History-MetarReports-row metar-20-20').text.strip()
    
    # Ardından "Çiyleşme Noktası" metnini bulun ve derece değerini çıkarın
    dew_point_cell = soup.find("tr", class_='table_no_border metar-20-50 History-MetarReports-row metar-20-20').find("td", {"class": "r i nowrap"}, string=re.compile("Çiyleşme Noktası"))
    degree_value = dew_point_cell.findNext("td").text.strip()

    # Derece değerini elde edin
    match = re.search(r'(\d+°C)', degree_value)
    if match:
        degree_value = match.group(1)
    else:
        degree_value = "NaN"

    result['20:20_nem'] = degree_value  # Farklı bir anahtar kullanarak derece değerini saklayın

except AttributeError:
    result['20:20'] = np.nan  # "20:20" metni bulunamazsa
    result['20:20_nem'] = "NaN"  # "Çiyleşme Noktası" metni veya derece değeri bulunamazsa
    
    
try:
    # İlk olarak "22:20" saatine ait metni bulun ve temizleyin
    result['22:20'] = soup.find('tr', class_='History-MetarReports-superseded metar-22-50 History-MetarReports-row metar-22-20').text.strip()
    
    # Ardından "Çiyleşme Noktası" metnini bulun ve derece değerini çıkarın
    dew_point_cell = soup.find("tr", class_='table_no_border metar-22-50 History-MetarReports-row metar-22-20').find("td", {"class": "r i nowrap"}, string=re.compile("Çiyleşme Noktası"))
    degree_value = dew_point_cell.findNext("td").text.strip()

    # Derece değerini elde edin
    match = re.search(r'(\d+°C)', degree_value)
    if match:
        degree_value = match.group(1)
    else:
        degree_value = "NaN"

    result['22:20_nem'] = degree_value  # Farklı bir anahtar kullanarak derece değerini saklayın

except AttributeError:
    result['22:20'] = np.nan  # "22:20" metni bulunamazsa
    result['22:20_nem'] = "NaN"  # "Çiyleşme Noktası" metni veya derece değeri bulunamazsa
    
     # Her URL için elde edilen sonuçları "all_results" listesine ekleyin
    all_results.append(result)

# Tüm sonuçları ekrana yazdırın
for i, result in enumerate(all_results, start=1):
    print(f"URL {i} Sonuçları:")
    for key, value in result.items():
        print(f"{key}: {value}")
    print()

URL 1 Sonuçları:
00:20: nan
00:20_nem: NaN
02:20: 02:20


24°C


1.013 mbar


18,5 km/sa, K


≥ 10,00 km


Bulut Tavanı ve Görüş Temiz
02:20_nem: 19°C
04:20: 04:20


24°C


1.013 mbar


5,56 km/sa, D


10,00 km


Parçalı Bulutlu (1.067 m)
04:20_nem: 18°C
06:20: nan
06:20_nem: NaN
08:20: 08:20


25°C


1.015 mbar


11,1 km/sa, K


10,00 km


Parçalı Bulutlu (1.067 m)
08:20_nem: 20°C
10:20: 10:20


28°C


1.015 mbar


14,8 km/sa, K


10,00 km


Parçalı Bulutlu (1.219 m)
10:20_nem: 17°C
12:20: nan
12:20_nem: NaN
14:20: nan
14:20_nem: NaN
16:20: 16:20


30°C


1.015 mbar


24,1 km/sa, K


≥ 10,00 km


Bulut Tavanı ve Görüş Temiz
16:20_nem: 17°C
18:20: nan
18:20_nem: NaN
20:20: 20:20


26°C


1.015 mbar


14,8 km/sa, KD


≥ 10,00 km


Bulut Tavanı ve Görüş Temiz
20:20_nem: 19°C
22:20: nan
22:20_nem: NaN



In [17]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np

# Her URL için sonuçları saklamak için bir liste oluşturun
all_results = []

# URL'leri döngü ile gezip sonuçları toplayın
for url in url_list:
    result = {}  # Her URL için yeni bir sonuç sözlüğü oluşturun

    # URL'yi getirin ve sayfanın içeriğini çekin
    headers = {
        "User-Agent": "Safari/537.36",
        "Referer": "https://tr.weatherspark.com/"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Her saat dilimini işlemek için bir döngü kullanın
    for hour in range(0, 24, 2):
        hour_str = f"{hour:02d}:20"
        
        try:
            # İlgili saat dilimine ait metni bulun ve temizleyin
            metar_row = soup.find('tr', class_=f'metar-{hour:02d}-50 History-MetarReports-row metar-{hour:02d}-20').text.strip()
            result[hour_str] = metar_row

            # "Çiyleşme Noktası" metnini bulun ve derece değerini çıkarın
            dew_point_cell = soup.find("tr", class_=f'table_no_border metar-{hour:02d}-50 History-MetarReports-row metar-{hour:02d}-20').find("td", {"class": "r i nowrap"}, string=re.compile("Çiyleşme Noktası"))
            degree_value = dew_point_cell.findNext("td").text.strip()

            # Derece değerini elde edin
            match = re.search(r'(\d+°C)', degree_value)
            if match:
                degree_value = match.group(1)
            else:
                degree_value = "NaN"

            result[f"{hour_str}_nem"] = degree_value  # Farklı bir anahtar kullanarak derece değerini saklayın

        except AttributeError:
            result[hour_str] = np.nan  # Metin bulunamazsa
            result[f"{hour_str}_nem"] = "NaN"  # "Çiyleşme Noktası" metni veya derece değeri bulunamazsa
    
    # Her URL için elde edilen sonuçları "all_results" listesine ekleyin
    all_results.append(result)

# Tüm sonuçları ekrana yazdırın
for i, result in enumerate(all_results, start=1):
    print(f"URL {i} Sonuçları:")
    for key, value in result.items():
        print(f"{key}: {value}")
    print()


URL 1 Sonuçları:
00:20: nan
00:20_nem: NaN
02:20: nan
02:20_nem: NaN
04:20: nan
04:20_nem: NaN
06:20: nan
06:20_nem: NaN
08:20: nan
08:20_nem: NaN
10:20: nan
10:20_nem: NaN
12:20: nan
12:20_nem: NaN
14:20: nan
14:20_nem: NaN
16:20: nan
16:20_nem: NaN
18:20: nan
18:20_nem: NaN
20:20: nan
20:20_nem: NaN
22:20: nan
22:20_nem: NaN

URL 2 Sonuçları:
00:20: nan
00:20_nem: NaN
02:20: nan
02:20_nem: NaN
04:20: nan
04:20_nem: NaN
06:20: nan
06:20_nem: NaN
08:20: nan
08:20_nem: NaN
10:20: nan
10:20_nem: NaN
12:20: nan
12:20_nem: NaN
14:20: nan
14:20_nem: NaN
16:20: nan
16:20_nem: NaN
18:20: nan
18:20_nem: NaN
20:20: nan
20:20_nem: NaN
22:20: nan
22:20_nem: NaN

URL 3 Sonuçları:
00:20: nan
00:20_nem: NaN
02:20: nan
02:20_nem: NaN
04:20: nan
04:20_nem: NaN
06:20: nan
06:20_nem: NaN
08:20: nan
08:20_nem: NaN
10:20: nan
10:20_nem: NaN
12:20: nan
12:20_nem: NaN
14:20: nan
14:20_nem: NaN
16:20: nan
16:20_nem: NaN
18:20: nan
18:20_nem: NaN
20:20: nan
20:20_nem: NaN
22:20: nan
22:20_nem: NaN

URL 4 Sonu

In [9]:
url_list

['https://tr.weatherspark.com/h/d/95434/2023/7/1/01-July-2023-tarihinde-in%C4%B0stanbul-T%C3%BCrkiye-Ortalama-Hava-Durumu#Figures-Temperature',
 'https://tr.weatherspark.com/h/d/95434/2023/7/2/02-July-2023-tarihinde-in%C4%B0stanbul-T%C3%BCrkiye-Ortalama-Hava-Durumu#Figures-Temperature',
 'https://tr.weatherspark.com/h/d/95434/2023/7/3/03-July-2023-tarihinde-in%C4%B0stanbul-T%C3%BCrkiye-Ortalama-Hava-Durumu#Figures-Temperature',
 'https://tr.weatherspark.com/h/d/95434/2023/7/4/04-July-2023-tarihinde-in%C4%B0stanbul-T%C3%BCrkiye-Ortalama-Hava-Durumu#Figures-Temperature',
 'https://tr.weatherspark.com/h/d/95434/2023/7/5/05-July-2023-tarihinde-in%C4%B0stanbul-T%C3%BCrkiye-Ortalama-Hava-Durumu#Figures-Temperature',
 'https://tr.weatherspark.com/h/d/95434/2023/7/6/06-July-2023-tarihinde-in%C4%B0stanbul-T%C3%BCrkiye-Ortalama-Hava-Durumu#Figures-Temperature',
 'https://tr.weatherspark.com/h/d/95434/2023/7/7/07-July-2023-tarihinde-in%C4%B0stanbul-T%C3%BCrkiye-Ortalama-Hava-Durumu#Figures-Temper

In [18]:
import pandas as pd

# all_results listesini kullanarak bir DataFrame oluşturun
df = pd.DataFrame(all_results)

In [19]:
df

,00:20,00:20_nem,02:20,02:20_nem,04:20,04:20_nem,06:20,06:20_nem,08:20,08:20_nem,...,14:20,14:20_nem,16:20,16:20_nem,18:20,18:20_nem,20:20,20:20_nem,22:20,22:20_nem
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"08:20\n\n\n26°C\n\n\n1.012 mbar\n\n\n18,5 km/s...",20°C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
headers = {
    "User-Agent": "Safari/537.36",
    "Referer": "https://tr.weatherspark.com/"
}

response = requests.get(url, headers=headers)


In [22]:
import re
import numpy as np

# Her URL için sonuçları saklamak için bir liste oluşturun
all_results = []

# URL'leri döngü ile gezip sonuçları toplayın
for url in url_list:
    result = {}  # Her URL için yeni bir sonuç sözlüğü oluşturun

    # URL'yi getirin ve sayfanın içeriğini çekin
    headers = {
        "User-Agent": "Safari/537.36",
        "Referer": "https://tr.weatherspark.com/"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    try:
        # İlgili saat dilimine ait metni bulun ve temizleyin
        result[f'00:20'] = soup.find('tr', class_=f'metar-00-50 History-MetarReports-row metar-00-20').text.strip()
        
        # "Çiyleşme Noktası" metnini bulun ve derece değerini çıkarın
        dew_point_cell = soup.find("tr", class_=f'table_no_border metar-00-50 History-MetarReports-row metar-00-20').find("td", {"class": "r i nowrap"}, string=re.compile("Çiyleşme Noktası"))
        degree_value = dew_point_cell.findNext("td").text.strip()

        # Derece değerini elde edin
        match = re.search(r'(\d+°C)', degree_value)
        if match:
            degree_value = match.group(1)
        else:
            degree_value = "NaN"

        result[f'00:20_nem'] = degree_value  # Farklı bir anahtar kullanarak derece değerini saklayın

    except AttributeError:
        result[f'00:20'] = np.nan  # Metin bulunamazsa
        result[f'00:20_nem'] = "NaN"  # "Çiyleşme Noktası" metni veya derece değeri bulunamazsa

# Sonuçları yazdırın veya istediğiniz şekilde işleyin
print(result)





{'00:20': nan, '00:20_nem': 'NaN'}
